### Feature extractor
Since the bad organization of the dataset, in order to have a more organized code, we used this file to create 3 .csv files with the following features:
* All the files are csv files where the columns are the elements of the exctracted (using the encoder part of the network) feature vector and the first column is the name of the related image;
* All the image in the sets trully exists in one of the image folders;
* Since no image for the test set exists we partitioned the train one.


In [1]:
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
import glob
from PIL import Image
from time import time
import pandas as pd
from keras import Input, layers
from keras import optimizers
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.preprocessing import image
from keras.utils import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout, Bidirectional, Add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.utils import to_categorical
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#use your path to download the dataset
!unzip /content/drive/MyDrive/Natural\ Language\ Processing/projectDataset

In [3]:
train_rad_ = pd.read_csv('/content/all_data/train/radiologytraindata.csv')
test_rad = train_rad_[0:10000]
train_rad = train_rad_[10000:].reset_index()
val_rad = pd.read_csv('/content/all_data/validation/radiologyvaldata.csv')
test_rad.drop(columns=['id'],inplace=True)
train_rad.drop(columns=['id'],inplace=True)
val_rad.drop(columns=['id'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
#Pretrained model for the featere vector extraction (encoder part)
model = InceptionV3(weights='imagenet')
model_new = Model(model.input, model.layers[-2].output)

96112376/96112376 [==============================] - 5s 0us/step


In [5]:
train_path = '/content/all_data/train/radiology/images/'
validation_path = '/content/all_data/validation/radiology/images/'
test_path = '/content/all_data/train/radiology/images/'

In [13]:
#Each image is preprocessed and then feeded to the pretrained network
def preprocess(image_path):
    if  os.path.exists(image_path):
      img=None
      try:
        img = load_img(image_path, target_size=(299, 299))
      except:
        print("image not recognized")
        return None
    else:
       return None
    x = None
    if img is not None:
      x = img_to_array(img)
      x = np.expand_dims(x, axis=0)
      x = preprocess_input(x)
    return x

def encode(image):
    image = preprocess(image) 
    fea_vec = None
    if image is not None:
      fea_vec = model_new.predict(image, verbose=0) 
      fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    return fea_vec

def get_features(path, dataframe):
  encoding = {}
  for i in range(len(dataframe)):
    try:
      img_name = dataframe['name'][i]
      img = path + img_name
      encoded_img = encode(img)
      if encoded_img is not None:
        encoding[img_name] = encoded_img
    except:
        print('err')
  return encoding

In [ ]:
encoding_train = get_features(train_path, train_rad)
pd.DataFrame(encoding_train.values()).to_csv("features_training.csv")
pd.DataFrame(encoding_train.keys()).to_csv("img_training.csv")

In [15]:
encoding_test = get_features(train_path, test_rad)
pd.DataFrame(encoding_test.values()).to_csv("features_test.csv")
pd.DataFrame(encoding_test.keys()).to_csv("img_tes.csv")

In [14]:
encoding_val = get_features(validation_path, val_rad)
pd.DataFrame(encoding_val.values()).to_csv("features_val.csv")
pd.DataFrame(encoding_val.keys()).to_csv("img_val.csv")